In [ ]:
import functions_framework
import csv
import requests
from urllib.parse import urlparse, parse_qs
import time
from google.cloud import storage




@functions_framework.http
def allp(request):
    urls = [
        "https://www.red.cl/restservice_v2/rest/conocerecorrido?codsint=124",
        "https://www.red.cl/restservice_v2/rest/conocerecorrido?codsint=205c",
        "https://www.red.cl/restservice_v2/rest/conocerecorrido?codsint=210",
        "https://www.red.cl/restservice_v2/rest/conocerecorrido?codsint=213e",
        "https://www.red.cl/restservice_v2/rest/conocerecorrido?codsint=712",
        "https://www.red.cl/restservice_v2/rest/conocerecorrido?codsint=F01",
        "https://www.red.cl/restservice_v2/rest/conocerecorrido?codsint=F01c",
        "https://www.red.cl/restservice_v2/rest/conocerecorrido?codsint=F03",
        "https://www.red.cl/restservice_v2/rest/conocerecorrido?codsint=F03c",
        "https://www.red.cl/restservice_v2/rest/conocerecorrido?codsint=F10",
        "https://www.red.cl/restservice_v2/rest/conocerecorrido?codsint=F10c",
        "https://www.red.cl/restservice_v2/rest/conocerecorrido?codsint=F11",
        "https://www.red.cl/restservice_v2/rest/conocerecorrido?codsint=F12",
        "https://www.red.cl/restservice_v2/rest/conocerecorrido?codsint=F12c",
        "https://www.red.cl/restservice_v2/rest/conocerecorrido?codsint=F13",
        "https://www.red.cl/restservice_v2/rest/conocerecorrido?codsint=F14",
        "https://www.red.cl/restservice_v2/rest/conocerecorrido?codsint=F16",
        "https://www.red.cl/restservice_v2/rest/conocerecorrido?codsint=F18",
        "https://www.red.cl/restservice_v2/rest/conocerecorrido?codsint=F29",
        "https://www.red.cl/restservice_v2/rest/conocerecorrido?codsint=F30n",
        "https://www.red.cl/restservice_v2/rest/conocerecorrido?codsint=F33"

    ]




    codigos_buscar = ["PF88", "PF725", "PF176", "PF94", "PF960", "PF512", "PF212"]




    max_retries = 3   # Número máximo de intentos
    retry_delay = 60   # Tiempo de espera entre intentos en segundos




    # Configurar el cliente de Google Cloud Storage
    storage_client = storage.Client()




    # Nombre del bucket y nombre del archivo en Cloud Storage
    bucket_name = "bucket_final01"
    blob_name = "all_recorridos.csv"




    try:
        # Obtener referencia al bucket
        bucket = storage_client.bucket(bucket_name)




        # Obtener referencia al blob en el bucket
        blob = bucket.blob(blob_name)




        headers = ["id", "cod", "pos_latitud", "pos_longitud", "name", "comuna", "servicio_id", "servicio_cod", "destino", "horarios"]




        with blob.open("w", newline="", encoding="utf-8") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(headers)




            for url in urls:
                intentos = 0
                while intentos < max_retries:
                    try:
                        response = requests.get(url)
                        response.raise_for_status()  # Lanza una excepción para errores HTTP




                        if response.status_code == 200:
                            data = response.json()
                            parsed_url = urlparse(url)
                            query_params = parse_qs(parsed_url.query)
                            codsint = query_params['codsint'][0]




                            for sentido in ["ida", "regreso"]:
                                paraderos = data.get(sentido, {}).get("paraderos", [])




                                rows = []
                                for paradero in paraderos:
                                    if paradero["codSimt"] in codigos_buscar:
                                        for servicio in paradero["servicios"]:
                                            if sentido in data and "horarios" in data[sentido]:
                                                horarios = data[sentido]["horarios"]
                                                horarios_texto = ", ".join(f"{h['tipoDia']}: {h['inicio']}-{h['fin']}" for h in horarios)
                                            else:
                                                horarios_texto = ""




                                            destino = servicio.get("destino", "")




                                            row = [
                                                paradero["id"],
                                                paradero["cod"],
                                                paradero["pos"][0],  # Latitud
                                                paradero["pos"][1],  # Longitud
                                                paradero["name"],
                                                paradero["comuna"],
                                                servicio["id"],
                                                servicio["cod"],
                                                destino,
                                                horarios_texto
                                            ]
                                            rows.append(row)




                                if rows:
                                    writer.writerows(rows)
                                    print(f"Se han encontrado {len(rows)} paraderos en '{sentido}' para la URL '{url}'.")
                                else:
                                    print(f"No se encontraron paraderos en '{sentido}' para la URL '{url}' con los códigos especificados.")




                            print(f"Los datos de la URL '{url}' han sido agregados al archivo.")
                            break  # Salir del bucle de reintentos si todo sale bien




                    except requests.exceptions.RequestException as e:
                        print(f"Error al realizar la solicitud a la API '{url}': {e}")




                    intentos += 1
                    print(f"Intento {intentos}/{max_retries}. Reintentando en {retry_delay} segundos...")
                    time.sleep(retry_delay)




                if intentos >= max_retries:
                    print(f"No se pudo obtener datos de la API '{url}' después de {max_retries} intentos.")




        print(f"Todos los datos han sido guardados en gs://{bucket_name}/{blob_name}")
        return f"Archivo guardado exitosamente en gs://{bucket_name}/{blob_name}"




    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        return f"Error: {e}"


REQUERIMENTS
functions-framework
requests
gcsfs
fsspec
pandas
pandas-gbq
google-cloud-storage
